In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, explode
import pyspark.sql.functions as f

In [ ]:
# # Extract
# spark = SparkSession.builder.appName("ETLPipeline").getOrCreate()
# df = spark.read.text("Word_data.txt")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/29 10:12:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("PostgreSQL_PySpark") \
    .config("spark.jars", "/opt/homebrew/lib/postgresql-42.6.0.jar") \
    .getOrCreate()

In [ ]:
df = spark.read.text("Word_data.txt")

In [3]:
df

DataFrame[value: string]

In [4]:
df.show()

+--------------------+
|               value|
+--------------------+
|This is a Japanes...|
|The team members ...|
|As the years pass...|
|If you don't like...|
|He was disappoint...|
|When he encounter...|
|Situps are a terr...|
|Toddlers feeding ...|
|Edith could decid...|
|Her daily goal wa...|
|Tomorrow will bri...|
|His son quipped t...|
|He wondered why a...|
|If my calculator ...|
|The hummingbird's...|
|He went on a whis...|
|This is the last ...|
|I come from a tri...|
|The delicious aro...|
|Weather is not tr...|
+--------------------+
only showing top 20 rows



In [5]:
display(df)

DataFrame[value: string]

# Data Transformation 

In [6]:
df2 = df.withColumn("splitedData", f.split("value"," "))
df2.show()

+--------------------+--------------------+
|               value|         splitedData|
+--------------------+--------------------+
|This is a Japanes...|[This, is, a, Jap...|
|The team members ...|[The, team, membe...|
|As the years pass...|[As, the, years, ...|
|If you don't like...|[If, you, don't, ...|
|He was disappoint...|[He, was, disappo...|
|When he encounter...|[When, he, encoun...|
|Situps are a terr...|[Situps, are, a, ...|
|Toddlers feeding ...|[Toddlers, feedin...|
|Edith could decid...|[Edith, could, de...|
|Her daily goal wa...|[Her, daily, goal...|
|Tomorrow will bri...|[Tomorrow, will, ...|
|His son quipped t...|[His, son, quippe...|
|He wondered why a...|[He, wondered, wh...|
|If my calculator ...|[If, my, calculat...|
|The hummingbird's...|[The, hummingbird...|
|He went on a whis...|[He, went, on, a,...|
|This is the last ...|[This, is, the, l...|
|I come from a tri...|[I, come, from, a...|
|The delicious aro...|[The, delicious, ...|
|Weather is not tr...|[Weather, 

In [7]:

df3 = df2.withColumn("words", explode("splitedData"))
df3.show()

+--------------------+--------------------+--------+
|               value|         splitedData|   words|
+--------------------+--------------------+--------+
|This is a Japanes...|[This, is, a, Jap...|    This|
|This is a Japanes...|[This, is, a, Jap...|      is|
|This is a Japanes...|[This, is, a, Jap...|       a|
|This is a Japanes...|[This, is, a, Jap...|Japanese|
|This is a Japanes...|[This, is, a, Jap...|    doll|
|The team members ...|[The, team, membe...|     The|
|The team members ...|[The, team, membe...|    team|
|The team members ...|[The, team, membe...| members|
|The team members ...|[The, team, membe...|    were|
|The team members ...|[The, team, membe...|    hard|
|The team members ...|[The, team, membe...|      to|
|The team members ...|[The, team, membe...|    tell|
|The team members ...|[The, team, membe...|   apart|
|The team members ...|[The, team, membe...|   since|
|The team members ...|[The, team, membe...|    they|
|The team members ...|[The, team, membe...|   

In [8]:
wordsDF = df3.select("words")
wordsDF.show()

+--------+
|   words|
+--------+
|    This|
|      is|
|       a|
|Japanese|
|    doll|
|     The|
|    team|
| members|
|    were|
|    hard|
|      to|
|    tell|
|   apart|
|   since|
|    they|
|     all|
|    wore|
|   their|
|    hair|
|      in|
+--------+
only showing top 20 rows



In [9]:
wordCount = wordsDF.groupBy("words").count()

In [10]:
wordCount.show()

+-----------+-----+
|      words|count|
+-----------+-----+
|   Tomorrow|    4|
|         If|    8|
|      leave|    4|
|      corny|    4|
|        day|    4|
|preoccupied|    4|
|       even|    8|
|      crazy|    4|
|    bananas|    4|
|     priest|    4|
|        did|    4|
|    whether|    4|
|     Having|    4|
|        I'm|    4|
|      crime|    4|
|  surprised|    4|
|      James|    4|
|      could|    8|
|        buy|    4|
|        him|    8|
+-----------+-----+
only showing top 20 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, explode
import pyspark.sql.functions as f

In [ ]:
# Load to PostgreSQL
driver = "org.postgresql.Driver"
url = "database-1.cvq2m462i7av.us-east-2.rds.amazonaws.com/"  # Add the database name
table = "kps_schema_pyspark.WordCount"
user = "postgres"
password = "abc12345678" 

In [ ]:
#Write the wordcount
wordCount.write.format("jdbc") \
    .option("driver", driver) \
    .option("url", url) \
    .option("dbtable", table) \
    .option("mode", "append") \
    .option("user", user) \
    .option("password", password) \
    .save()

In [ ]:


# # Set up the Spark session
# spark = SparkSession.builder \
#     .appName("ETL Pipeline") \
#     .config("spark.jars", "/path/to/postgresql-42.6.0.jar")  # Ensure the JDBC driver is added here
#     .getOrCreate()

# # Extract data (Make sure you specify the correct path for the file)
# file_path = "./WordData.txt"  # Replace with the actual path to the file
# df = spark.read.text(file_path)

# # Transformation
# df2 = df.withColumn("splitedData", f.split("value", " "))  # Split the data by space
# df3 = df2.withColumn("words", explode("splitedData"))  # Explode the array to individual words
# wordsDF = df3.select("words")
# wordCount = wordsDF.groupBy("words").count()  # Count the occurrences of each word



# wordCount.write.format("jdbc") \
#     .option("driver", driver) \
#     .option("url", url) \
#     .option("dbtable", table) \
#     .option("mode", "append") \
#     .option("user", user) \
#     .option("password", password) \
#     .save()
